In [ ]:
import itertools
import collections

In [ ]:
from pathlib import Path

In [ ]:
MAX_BITS = 64
TOTAL_BITS = 72

In [ ]:
def get_bit_layouts(num_fields, max_bits, total_bits):
    q = collections.deque([()])
    item = q.popleft()
    while len(item) < num_fields:
        upper_bound = min(total_bits - sum(item), max_bits)
        if len(item) == num_fields - 1:
            q.append(item + (upper_bound,))
        else:
            lower_bound = max(0, total_bits - sum(item) - (num_fields - len(item) - 1) * max_bits)
            q.extend([item + (x,) for x in range(lower_bound, upper_bound + 1)])
        item = q.popleft()
    q.appendleft(item)
    return q

In [ ]:
bit_layouts_all = list(get_bit_layouts(3, MAX_BITS, TOTAL_BITS))
bit_layouts = list(filter(lambda bl: (31 in bl) or (32 in bl) or (62 in bl) or (63 in bl) or (64 in bl), bit_layouts_all))

In [ ]:
len(bit_layouts)

In [ ]:
len(set(bit_layouts))

In [ ]:
set(map(sum, bit_layouts))

In [ ]:
def fill_zeros(n):
    return '0' * n
def fill_ones(n):
    return '1' * n

def get_fill_repeat(seq):
    len_seq = len(seq)
    return (lambda n: seq * (n // len_seq) + seq[:(n % len_seq if n >= 0 else 0)])
def get_fill_prefix_and_existing(prefix, func):
    len_prefix = len(prefix)
    return (lambda n: prefix[:max(0, n)] + func(n - len_prefix))
def get_fill_existing_and_suffix(func, suffix):
    len_suffix = len(suffix)
    return (lambda n: func(n - len_suffix) + (suffix[-n:] if n >= 1 else ''))


def get_fill_zero_and_existing(func):
    return (lambda n: ('0' if n >= 1 else '') + func(n - 1))
def get_fill_one_and_existing(func):
    return (lambda n: ('1' if n >= 1 else '') + func(n - 1))

def get_fill_existing_and_zero(func):
    return (lambda n: func(n - 1) + ('0' if n >= 1 else ''))
def get_fill_existing_and_one(func):
    return (lambda n: func(n - 1) + ('1' if n >= 1 else ''))

In [ ]:
basic_patterns = [fill_zeros, fill_ones]
transformers = [get_fill_zero_and_existing, get_fill_one_and_existing, get_fill_existing_and_zero, get_fill_existing_and_one]
derived_patterns = [
    fill_zeros,
    get_fill_prefix_and_existing('00', fill_ones),
    get_fill_prefix_and_existing('0', fill_ones),
    get_fill_repeat('10'),
    get_fill_repeat('01'),
    get_fill_repeat('1000''0000'),
    get_fill_prefix_and_existing('1', fill_zeros),
    fill_ones,
]
# derived_patterns = list(itertools.chain.from_iterable([map(func, basic_patterns) for func in transformers]))

In [ ]:
test_values = [-1, 0, 1, 2, 3, 6, 16]
results = []
for func in derived_patterns:
    results.append({'pattern': func, 'results': [func(n) for n in test_values]})

In [ ]:
results

In [ ]:
fillings_by_num_bits = {n: set([func(n) for func in derived_patterns]) for n in range(0, MAX_BITS + 1)}

In [ ]:
fillings_by_num_bits

In [ ]:
bit_layouts

In [ ]:
target_dir = Path('R:\\Temp')
target_dir_ksy = target_dir / 'ksy'
target_dir_kst = target_dir / 'kst'
target_dir_bin = target_dir / 'bin'

In [ ]:
for t_dir in (target_dir_ksy, target_dir_kst, target_dir_bin):
    t_dir.mkdir(exist_ok=True)
    if t_dir in (target_dir_ksy, target_dir_kst):
        for bit_endian in ('be', 'le'):
            (t_dir / bit_endian).mkdir(exist_ok=True)

In [ ]:
assert TOTAL_BITS % 8 == 0, 'TOTAL_BITS ({:d}) not divisible by 8'.format(TOTAL_BITS)

In [ ]:
num_tests = 0

for bit_layout in bit_layouts:
    ksy_id = ''.join(map(lambda n: f'b{n:d}', bit_layout))
    layout_data = itertools.product(*[enumerate(fillings_by_num_bits[n]) for n in bit_layout])

    ksy_seq = ''
    for j, bits in enumerate(bit_layout):
        ksy_seq += f"""\
  - id: {chr(ord('a') + j)}
    type: b{bits:d}
"""

    for bit_endian in ('be', 'le'):
        ksy_code = f"""\
meta:
  id: {ksy_id}
  bit-endian: {bit_endian}
seq:
"""
        ksy_code += ksy_seq
        with open(target_dir_ksy / bit_endian / f'{ksy_id}.ksy', 'w', encoding='utf-8') as f_ksy:
            f_ksy.write(ksy_code)

    for val_pairs in layout_data:
        num_tests += 1
        filling_id_parts, vals = zip(*val_pairs)
        test_id = ksy_id +  '_v' + 'x'.join(map(str, filling_id_parts))

        kst_asserts = ''
        for j, val in enumerate(vals):
            val_int = int(val, 2) if val != '' else 0
            if len(val) == 1:
                val = ('false', 'true')[val_int]
            else:
                val = f'0x{val_int:_x}' if val_int != 0 else '0'
            kst_asserts += f"""\
  - actual: {chr(ord('a') + j)}
    expected: {val}
"""

        for bit_endian in ('be', 'le'):
            payload = \
                int(''.join(reversed(vals) if bit_endian == 'le' else vals), 2) \
                .to_bytes(TOTAL_BITS // 8, byteorder=('little' if bit_endian == 'le' else 'big'))
            payload_fname = payload.hex('_') + '.bin'
            try:
                with open(target_dir_bin / payload_fname, 'xb') as f_bin:
                    f_bin.write(payload)
            except FileExistsError:
                pass

            kst_code = f"""\
id: {ksy_id}
data: {payload_fname}
asserts:
"""
            kst_code += kst_asserts

            kst_dir = target_dir_kst / bit_endian
            with open(kst_dir / f'{test_id}.kst', 'w', encoding='utf-8') as f_kst:
                f_kst.write(kst_code)

In [ ]:
print('{:_d}'.format(num_tests))

In [ ]:
estimated_bin_size = 256
estimated_ksy_size = 256
estimated_kst_size = 256
estimated_spec_size = 256
num_targets = 12

In [ ]:
estimated_size_total = len(test_payloads) * estimated_bin_size + len(test_data) * (estimated_ksy_size + estimated_kst_size + num_targets * estimated_spec_size)

In [ ]:
print('{:_d}'.format(estimated_size_total))

In [ ]:
print(len(bit_layouts) / 256)